In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [11]:
def find_reflection_points(line, reflect_pts=None ):
    
    def check_reflection(line, idx):
        front_ref = line[:idx][::-1]
        back = line[idx:]
        s_len = min( len(front_ref), len(back))
        return front_ref[:s_len] == back[:s_len]
    
    if reflect_pts is None:
        reflect_pts = []
        for i in range(1,len(line)):
            if check_reflection(line,i):
                reflect_pts.append(i)
        return reflect_pts
    new_pts =[]
    while len(reflect_pts) > 0 : 
        i = reflect_pts.pop()
        if check_reflection(line,i):
            new_pts.append(i)
    return new_pts
        

In [17]:
def process_image(image_grid):
    # check for vertical reflection
    reflect_pts = find_reflection_points( image_grid[0]) 
    if len(reflect_pts) > 0 :
        for row in image_grid[1:]:
            reflect_pts = find_reflection_points(row, reflect_pts)
            if len(reflect_pts) < 1 :
                break
    
    #check horizontal
    
    col=''.join([ r[0] for r in image_grid])
    col_pts = find_reflection_points(col ) 
    if len(col_pts) > 0 :
        for c in range(1,len(image_grid[0])):
            col=''.join([ r[c] for r in image_grid])
            col_pts = find_reflection_points(col, col_pts)
            if len(col_pts) < 1 :
                break
    score =0
#    print(reflect_pts, col_pts)
    for c in reflect_pts:
        score+=c
    for r in col_pts:
        score+=100*r
    return score

In [33]:
def locate_differences(line, ref):
    front_ref = line[:ref][::-1]
    back = line[ref:]
    s_len = min( len(front_ref), len(back))
    dif = sum( [ i!=j for i,j in zip(front_ref[:s_len],back[:s_len]) ])
    return dif == 1

def find_smudge(image_grid):
    # check for vertical reflection
    r_counts = {}
    for ir, row in enumerate(image_grid):
        reflect_pts = find_reflection_points(row)
        for r in reflect_pts:
            try: 
                r_counts[r].append(ir)
            except:
                r_counts[r] =[ir]
    new_reflect=[]
    r_total = len(image_grid)
    for ref, rows in r_counts.items():
        if len(rows) == r_total-1:
            missing = set(range(r_total))-set(rows)
            ridx = list(missing)[0]
            if locate_differences( image_grid[ridx], ref):
                new_reflect.append(ref)
            
    
    #check horizontal
    c_counts={}
    for c in range(len(image_grid[0])):
        col=''.join([ r[c] for r in image_grid])
        col_pts = find_reflection_points(col)
        for ic in col_pts:
            try: 
                c_counts[ic].append(c)
            except:
                c_counts[ic] =[c]
                
    col_reflect=[]
    c_total = len(image_grid[0])
    for ref, cols in c_counts.items():
        if len(cols) == c_total-1:
            missing = set(range(c_total))-set(cols)
            cidx = list(missing)[0]
            
            col=''.join([ r[cidx] for r in image_grid])
            
            if locate_differences( col, ref):
                col_reflect.append(ref)
        
    score =0
#    print(reflect_pts, col_pts)
    for c in new_reflect:
        score+=c
    for r in col_reflect:
        score+=100*r
    return score

In [34]:
def do_part_one(file_path):
    
    images=[]
    with open(file_path,'r') as f:
        image =[]
        for line in f.readlines():
            line =line.strip()
            if len(line) < 2:
                images.append(image.copy())
                image = []
            else:
                image.append(line)
        images.append(image)
        
    total = 0
    for im in images:
        total+=process_image(im)
    return total

In [19]:
%%time
do_part_one('input_data/day_13.txt')

CPU times: user 12.9 ms, sys: 1.96 ms, total: 14.8 ms
Wall time: 13 ms


27505

# Part 2

In [35]:
def do_part_two(file_path):
    
    images=[]
    with open(file_path,'r') as f:
        image =[]
        for line in f.readlines():
            line =line.strip()
            if len(line) < 2:
                images.append(image.copy())
                image = []
            else:
                image.append(line)
        images.append(image)
        
    total = 0
    for im in images:
        total+=find_smudge(im)
    return total

In [37]:
%%time
do_part_two('input_data/day_13.txt')

CPU times: user 34.8 ms, sys: 2.59 ms, total: 37.4 ms
Wall time: 37.4 ms


22906